In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import xml.etree.ElementTree as ET 
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

Dataset setup/transformation

In [ ]:
from google.colab import drive
# drive_path = "/content/drive/MyDrive/ECGR 4106/4106 Final Proj"
drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/4106FinalProj"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# set to false if all data has been transformed
TRANSFORM_DATA = False #@param ["True", "False"] {type:"raw"}

In [ ]:
input_data = drive_path
annotations_path = drive_path + "/annotations"
images_path = drive_path + "/images"
output_data = drive_path + '/output'

In [ ]:
dataset = {
            "file":[],
            "name":[],    
            "width":[],
            "height":[],
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
           }

for anno in glob.glob(annotations_path+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]

In [ ]:
df=pd.DataFrame(dataset)
df.head()

,file,name,width,height,xmin,ymin,xmax,ymax
0,maksssksksss101,with_mask,301,400,48,294,164,400
1,maksssksksss103,with_mask,400,300,42,54,94,110
2,maksssksksss103,with_mask,400,300,188,46,236,106
3,maksssksksss103,with_mask,400,300,261,88,303,130
4,maksssksksss103,with_mask,400,300,368,112,400,151


In [ ]:
name_dict = {
    'with_mask': 0,
    'mask_weared_incorrect': 1,
    'without_mask': 2,
}

df['class'] = df['name'].map(name_dict)

In [ ]:
fileNames = [*os.listdir(images_path)]
print('There are {} images in the dataset'.format(len(fileNames)))

There are 853 images in the dataset


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(fileNames, test_size=0.1, random_state=42)
test, val = train_test_split(test, test_size=0.7, random_state=42)
print("Length of Train =",len(train))
print("Length of Valid =",len(val))
print("Length of test =", len(test))

Length of Train = 767
Length of Valid = 61
Length of test = 25


In [ ]:
sizeX = 256
sizeY = 256

if TRANSFORM_DATA:
  if not os.path.isdir(output_data + "/yolov4"):
    os.chdir(output_data )
    os.makedirs('./yolov4/data/train')
    os.makedirs('./yolov4/data/val')
    os.makedirs('./yolov4/data/test')
    os.makedirs('./yolov4/data/train/images')
    os.makedirs('./yolov4/data/train/labels')
    os.makedirs('./yolov4/data/test/images')
    os.makedirs('./yolov4/data/test/labels')
    os.makedirs('./yolov4/data/val/images')
    os.makedirs('./yolov4/data/val/labels')
  
  from PIL import Image

  def copyImages(imageList, folder_Name):
      for image in imageList:
          img = Image.open(input_data+"/images/"+image)
          img1 = img.resize((sizeX, sizeY))
          _ = img1.save(output_data+"/yolov4/data/"+folder_Name+"/images/"+image)

  copyImages(train, "train")
  copyImages(val, "val")
  copyImages(test, "test")



In [ ]:
df['xmax'] = (sizeX/df['width'])*df['xmax']
df['ymax'] = (sizeY/df['height'])*df['ymax']
df['xmin'] = (sizeX/df['width'])*df['xmin']
df['ymin'] = (sizeY/df['height'])*df['ymin']

df[['xmax', 'ymax', 'xmin', 'ymin']] = df[['xmax', 'ymax', 'xmin', 'ymin']].astype('int64')

In [ ]:
df['x_center'] = (df['xmax']+df['xmin']) / (2*sizeX)
df['y_center'] = (df['ymax']+df['ymin']) / (2*sizeY)
df['box_height'] = (df['xmax']-df['xmin']) / (sizeX)
df['box_width'] = (df['ymax']-df['ymin']) / (sizeY)

In [ ]:
df = df.astype('string')
df.head()

,file,name,width,height,xmin,ymin,xmax,ymax,class,x_center,y_center,box_height,box_width
0,maksssksksss101,with_mask,301,400,40,188,139,256,0,0.349609375,0.8671875,0.38671875,0.265625
1,maksssksksss103,with_mask,400,300,26,46,60,93,0,0.16796875,0.271484375,0.1328125,0.18359375
2,maksssksksss103,with_mask,400,300,120,39,151,90,0,0.529296875,0.251953125,0.12109375,0.19921875
3,maksssksksss103,with_mask,400,300,167,75,193,110,0,0.703125,0.361328125,0.1015625,0.13671875
4,maksssksksss103,with_mask,400,300,235,95,256,128,0,0.958984375,0.435546875,0.08203125,0.12890625


In [ ]:
if TRANSFORM_DATA:
  def create_labels(image_list, data_name):
      fileNames = [x.split(".")[0] for x in image_list]

      for name in fileNames:
          data = df[df.file==name]
          box_list = []
          
          for index in range(len(data)):
              row = data.iloc[index]
              box_list.append(row['class']+" "+row["x_center"]+" "+row["y_center"]\
                          +" "+row["box_height"]+" "+row["box_width"])
              
          text = "\n".join(box_list)
          with open(output_data+"/yolov4/data/"+data_name+"/labels/"+name+".txt", "w") as file:
              file.write(text)


  create_labels(train, "train")
  create_labels(val, "val")
  create_labels(test, "test")

YOLO v5


In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr /content/yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!pip install yolov5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from IPython.display import Image, clear_output
import torch
from yolov5 import utils
display = utils.notebook_init()

YOLOv5 🚀 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.4/166.8 GB disk)


In [ ]:
# configure .yaml file to guide the model for training
yaml_text = """train: /content/drive/MyDrive/4106FinalProj/output/yolov4/data/train/images
val: /content/drive/MyDrive/4106FinalProj/output/yolov4/data/train/images

nc: 3
names: ['with_mask', 'mask_weared_incorrect', 'without_mask']"""

with open("/content/yolov5/data/data.yaml", 'w') as file:
    file.write(yaml_text)

%cat /content/yolov5/data/data.yaml

train: /content/drive/MyDrive/4106FinalProj/output/yolov4/data/train/images
val: /content/drive/MyDrive/4106FinalProj/output/yolov4/data/train/images

nc: 3
names: ['with_mask', 'mask_weared_incorrect', 'without_mask']

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Hyper Parameters

In [ ]:
%%writetemplate /content/yolov5/models/hyp.yaml

optimizer: 'adam'
lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3) 
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf) 
momentum: 0.937  # SGD momentum/Adam beta1 
weight_decay: 0.0005  # optimizer weight decay 5e-4 
warmup_epochs: 3.0  # warmup epochs (fractions ok) 
warmup_momentum: 0.8  # warmup initial momentum 
warmup_bias_lr: 0.1  # warmup initial bias lr 
box: 0.05  # box loss gain 
cls: 0.5  # cls loss gain 
cls_pw: 1.0  # cls BCELoss positive_weight 
obj: 1.0  # obj loss gain (scale with pixels) 
obj_pw: 1.0  # obj BCELoss positive_weight 
iou_t: 0.20  # IoU training threshold 
anchor_t: 4.0  # anchor-multiple threshold 
# anchors: 3  # anchors per output layer (0 to ignore) 
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5) 
hsv_h: 0.015  # image HSV-Hue augmentation (fraction) 
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction) 
hsv_v: 0.4  # image HSV-Value augmentation (fraction) 
degrees: 15.0 # image rotation (+/- deg) 
translate: 0.1  # image translation (+/- fraction) 
scale: 0.5  # image scale (+/- gain) 
shear: 0.0  # image shear (+/- deg) 
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001 
flipud: 0.0  # image flip up-down (probability) 
fliplr: 0.5  # image flip left-right (probability) 
mosaic: 1.0  # image mosaic (probability) 
mixup: 0.0  # image mixup (probability) 
copy_paste: 0.0  # segment copy-paste (probability) 

YOLO V5 Model

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: 3  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.5  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

YOLO V3 Model

In [ ]:
# %%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# # Parameters
# nc: 3  # number of classes
# depth_multiple: 0.33  # model depth multiple
# width_multiple: 0.5  # layer channel multiple
# anchors:
#   - [10,13, 16,30, 33,23]  # P3/8
#   - [30,61, 62,45, 59,119]  # P4/16
#   - [116,90, 156,198, 373,326]  # P5/32

# # darknet53 backbone
# backbone:
#   # [from, number, module, args]
#   [[-1, 1, Conv, [32, 3, 1]],  # 0
#    [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
#    [-1, 1, Bottleneck, [64]],
#    [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
#    [-1, 2, Bottleneck, [128]],
#    [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
#    [-1, 8, Bottleneck, [256]],
#    [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
#    [-1, 8, Bottleneck, [512]],
#    [-1, 1, Conv, [1024, 3, 2]],  # 9-P5/32
#    [-1, 4, Bottleneck, [1024]],  # 10
#   ]

# # YOLOv3 head
# head:
#   [[-1, 1, Bottleneck, [1024, False]],
#    [-1, 1, Conv, [512, 1, 1]],
#    [-1, 1, Conv, [1024, 3, 1]],
#    [-1, 1, Conv, [512, 1, 1]],
#    [-1, 1, Conv, [1024, 3, 1]],  # 15 (P5/32-large)

#    [-2, 1, Conv, [256, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 8], 1, Concat, [1]],  # cat backbone P4
#    [-1, 1, Bottleneck, [512, False]],
#    [-1, 1, Bottleneck, [512, False]],
#    [-1, 1, Conv, [256, 1, 1]],
#    [-1, 1, Conv, [512, 3, 1]],  # 22 (P4/16-medium)

#    [-2, 1, Conv, [128, 1, 1]],
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 6], 1, Concat, [1]],  # cat backbone P3
#    [-1, 1, Bottleneck, [256, False]],
#    [-1, 2, Bottleneck, [256, False]],  # 27 (P3/8-small)

#    [[27, 22, 15], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5)
#   ]

In [ ]:
start = datetime.now()
!python /content/yolov5/train.py --img 256 --batch 32 --epochs 200 --data /content/yolov5/data/data.yaml --cfg /content/yolov5/models/custom_yolov5s.yaml --hyp /content/yolov5/models/hyp.yaml --name yolov5s_results  --cache
end = datetime.now()

train: weights=yolov5/yolov5s.pt, cfg=/content/yolov5/models/custom_yolov5s.yaml, data=/content/yolov5/data/data.yaml, hyp=/content/yolov5/models/hyp.yaml, epochs=200, batch_size=32, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-251-gc23a441 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: optimizer=adam, lr0=0.001, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5

In [ ]:
print("Runtime =",end-start)

Runtime = 0:29:58.563508


In [ ]:
!python /content/yolov5/detect.py --source /content/drive/MyDrive/4106FinalProj/output/yolov4/data/test/images/ --weight /content/yolov5/runs/train/yolov5s_results4/weights/best.pt --name expTestImage --imgsz 256 --conf 0.4  --line-thickness 1

detect: weights=['/content/yolov5/runs/train/yolov5s_results4/weights/best.pt'], source=/content/drive/MyDrive/4106FinalProj/output/yolov4/data/test/images/, data=yolov5/data/coco128.yaml, imgsz=[256, 256], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=expTestImage, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-251-gc23a441 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
custom_YOLOv5s summary: 213 layers, 7018216 parameters, 0 gradients
image 1/25 /content/drive/.shortcut-targets-by-id/1ipTchgzl4oHKO3-iBYAJYpRcMVTCYzEp/4106FinalProj/output/yolov4/data/test/images/maksssksksss160.png: 256x256 2 with_masks, Done. (0.008s)
image 2/25 /content/drive/.shortcut-targets-by-id/1ipTchgzl4oHKO3

In [ ]:
color_dict = {
    'with_mask': (255, 0, 0),
    'mask_weared_incorrect':  (0, 255, 0),
    'without_mask': (0, 0, 255) 
}

In [ ]:
def show_image(img_id):
    df_image = df[df.file==img_id]
    df_image[['xmin', 'ymin', 'xmax', 'ymax']] = df_image[['xmin', 'ymin', 'xmax', 'ymax']].astype('int64')
    path = output_data + '/yolov4/data/test/images/'+img_id# +'.png'
    img = plt.imread(path)

    imge = img.copy()

    for index in range(len(df_image)):
        row = df_image.iloc[index]
        cv2.rectangle(imge, 
                      (row['xmin'], row['ymin']),
                      (row['xmax'], row['ymax']),
                      color=color_dict[row['name']],
                      thickness=1)

    img_pred = plt.imread('/content/yolov5/runs/detect/expTestImage5/'+img_id)
    # ===================================
    plt.figure(figsize=(14,17))

    plt.subplot(1,2,1)
    plt.imshow(imge)
    plt.axis('off')
    plt.title('Image with Truth Box')

    plt.subplot(1,2,2)
    plt.imshow(img_pred)
    plt.axis('off')
    plt.title('Image with Predicted Box')

In [ ]:
import os, random
show_image(random.choice(os.listdir(output_data + "/yolov4/data/test/images/"))) 
show_image(random.choice(os.listdir(output_data + "/yolov4/data/test/images/"))) 
plt.show()

Video Detection

In [ ]:
!python /content/yolov5/detect.py --source /content/drive/MyDrive/4106FinalProj/vid0.mp4 --weight /content/yolov5/runs/train/yolov5s_results4/weights/best.pt --name expTestImage --imgsz 256 --conf 0.4  --line-thickness 1

detect: weights=['/content/yolov5/runs/train/yolov5s_results4/weights/best.pt'], source=/content/drive/MyDrive/4106FinalProj/vid0.mp4, data=yolov5/data/coco128.yaml, imgsz=[256, 256], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=expTestImage, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-251-gc23a441 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
custom_YOLOv5s summary: 213 layers, 7018216 parameters, 0 gradients
video 1/1 (1/465) /content/drive/.shortcut-targets-by-id/1ipTchgzl4oHKO3-iBYAJYpRcMVTCYzEp/4106FinalProj/vid0.mp4: 256x256 1 with_mask, Done. (0.013s)
video 1/1 (2/465) /content/drive/.shortcut-targets-by-id/1ipTchgzl4oHKO3-iBYAJYpRcMVTCYzEp/4106FinalProj/vid0.mp4: 256x256 1

Download Model

In [ ]:
!zip -r /content/file.zip /content/yolov5/runs/

from google.colab import files
files.download("/content/file.zip")

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5s_results/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5s_results/hyp.yaml (deflated 43%)
  adding: content/yolov5/runs/train/yolov5s_results/weights/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5s_results/opt.yaml (deflated 47%)
  adding: content/yolov5/runs/train/yolov5s_results/events.out.tfevents.1655079592.e47f431ab96e.1786.0 (deflated 5%)
  adding: content/yolov5/runs/train/yolov5s_results4/ (stored 0%)
  adding: content/yolov5/runs/train/yolov5s_results4/P_curve.png (deflated 10%)
  adding: content/yolov5/runs/train/yolov5s_results4/results.csv (deflated 83%)
  adding: content/yolov5/runs/train/yolov5s_results4/hyp.yaml (deflated 43%)
  adding: content/yolov5/runs/train/yolov5s_results4/labels.jpg (deflated 18%)
  adding: content/yolov5/runs/train/yolov5s_results4/val_batch2_labels.jpg (deflated 9%)
  adding: content/yolov5/runs/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>